In [ ]:
#BASIC

CREATE TABLE new AS
select 
t1.id, 
t2.id as matches_id,
t1.team_long_name
FROM teams_spain t1
INNER JOIN matches_spain t2 on (t1.id=t2.id);

In [ ]:
#JOIN

SELECT teams_spain.id as teams_id, matches_spain.id as matches_id
FROM teams_spain
INNER JOIN matches_spain ON teams_spain.id=matches_spain.id;

In [ ]:
#REPLACE

UPDATE table name
SET column= 'cas'
WHERE column= 'cas.'

In [ ]:
#COUNT WITH CONDITION
SELECT COUNT(column)
FROM table
WHERE column<10;

In [ ]:
#DISTINCT AND COUNT 
SELECT DISTINCT column1, column2, ...
FROM table_name;

SELECT COUNT(DISTINCT Country) FROM Customers;

In [ ]:
#GROUP BY AND ORDER BY
SELECT column_name(s)
FROM table_name
WHERE condition
GROUP BY column_name(s)
ORDER BY column_name(s);

In [ ]:
# AVG, SUM, MIN, MAX
SELECT AVG(column_name)
FROM table_name
WHERE condition;

SELECT SUM(column_name)
FROM table_name
WHERE condition;

SELECT MIN(column_name)
FROM table_name
WHERE condition;

In [ ]:
#CONDITIONS
CREATE TABLE name_table distribute by hash(idd_2017) as
select
idd,
n_lloc
from table
where n_lloc_cat_2017 in ('Gestió activa') AND (n_lloc_cat_anterior not in ('Gestió activa') or n_lloc_cat_anterior is null);



SELECT * FROM Customers
WHERE Country='Germany' AND (City='Berlin' OR City='München');

In [ ]:
#AND

SELECT * FROM Customers
WHERE Country='Germany' AND City='Berlin';

In [ ]:
#OR

SELECT * FROM Customers
WHERE City='Berlin' OR City='München';

In [ ]:
#NOT

SELECT * FROM Customers
WHERE NOT Country='Germany';


SELECT * FROM Customers
WHERE NOT Country='Germany' AND NOT Country='USA';

In [ ]:
-- Select matches where Barcelona was the away team
SELECT  
	m.date,
	t.team_long_name AS opponent,
	CASE WHEN m.home_goal > m.away_goal THEN 'Barcelona win!'
        WHEN m.home_goal < m.away_goal THEN 'Barcelona loss :(' 
        ELSE 'Tie' END AS outcome
FROM matches_spain AS m
-- Join teams_spain to matches_spain
LEFT JOIN teams_spain AS t 
ON m.hometeam_id = t.team_api_id
WHERE m.awayteam_id = 8634;

In [ ]:
SELECT 
	date,
	-- Identify the home team as Barcelona or Real Madrid
	case when hometeam_id = 8634 THEN 'FC Barcelona' 
        else 'Real Madrid CF' END AS home,
    -- Identify the away team as Barcelona or Real Madrid
	case when awayteam_id = 8634 THEN 'FC Barcelona' 
        else 'Real Madrid CF' END AS away
FROM matches_spain
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);

In [ ]:
# CASE WHEN FILTERING NULLS

-- Select the season, date, home_goal, and away_goal columns
SELECT 
	season,
	date,
	home_goal,
	away_goal
FROM matches_italy
WHERE
-- Exclude games not won by Bologna
	CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
         WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
         END IS NOT NULL;

### Group by and count using case when



In [ ]:
SELECT 
	c.name AS country,
    -- Count matches in each of the 3 seasons
	COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
	COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
	COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY c.name;

### Sum the number of id that match the condition and group by country

In [ ]:
SELECT 
	c.name AS country,
    -- Sum the total records in each season where the home team won
	SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2012_2013,
 	SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal  
        THEN 1 ELSE 0 END) AS matches_2013_2014,
	SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY c.name;

### AVG  

In [ ]:
# COUNT THE NUMBER OF HOME WINS, AWAY WINS AND TIES IN EACH COUNTRY

SELECT 
    c.name AS country,
    -- Count the home wins, away wins, and ties in each country
	COUNT(case when m.home_goal > m.away_goal THEN m.id 
        END) AS home_wins,
	COUNT(case when m.home_goal < m.away_goal THEN m.id 
        END) AS away_wins,
	COUNT(case when m.home_goal = m.away_goal THEN m.id 
        END) AS ties
FROM country AS c
LEFT JOIN matches AS m
ON c.id = m.country_id
GROUP BY country;

In [ ]:
# COUNT THE AVERAGE OF TIMES THEY TIED

SELECT 
	c.name AS country,
    -- Calculate the percentage of tied games in each season
	AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
			WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
			END) AS ties_2013_2014,
	AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
			WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
			END) AS ties_2014_2015
FROM country AS c
LEFT JOIN matches AS m
ON c.id = m.country_id
GROUP BY country;

In [ ]:
#THE SAME BUT ROUNDING

SELECT 
	c.name AS country,
    -- Round the percentage of tied games to 2 decimal points
	ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
			 END),2) AS pct_ties_2013_2014,
	ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
			 END),2) AS pct_ties_2014_2015
FROM country AS c
LEFT JOIN matches AS m
ON c.id = m.country_id
GROUP BY country;

In [ ]:
#FILTER BY 2 TEAMS AND GROUP BY ALL THE ROWS SO WE CAN COUNT HOW MANY TIMES EACH TEAM APPEARS

-- Identify the home team as Bayern Munich, Schalke 04, or neither
SELECT 
	CASE WHEN hometeam_id = '10189' THEN 'FC Schalke 04'
        WHEN hometeam_id = '9823' THEN 'FC Bayern Munich'
         ELSE 'Other' END AS home_team,
	COUNT(id) AS total_matches
FROM matches_germany
-- Group by the CASE statement alias
GROUP BY home_team;

In [ ]:
# JOIN TABLES BY ITSELF IN ORDER TO ELIMINATE DUPLICATES AND CALCULATING PERC GROWTH
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM populations AS p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = p2.year - 5;

In [ ]:
# CATEGORISE VARIABLE AND FILTER BY YEAR IN THE SAME SCRIPT

SELECT country_code, size, year,
    -- 1. First case
    CASE WHEN populations.size > 50000000 THEN 'large'
        -- 2. Second case
        WHEN populations.size > 1000000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS popsize_group
-- 5. From table
FROM populations
-- 6. Focus on 2015
WHERE year = 2015;

In [ ]:
#CREATE NEW TABLE WITH ALL QUERIES FROM ANOTHER TABLE, INCLUDING CATEGORISING AND FILTERING

SELECT country_code, size,
    CASE WHEN size > 50000000 THEN 'large'
        WHEN size > 1000000 THEN 'medium'
        ELSE 'small' END
        AS popsize_group
-- 1. Into table
INTO pop_plus
FROM populations
WHERE year = 2015;

-- 2. Select all columns of pop_plus
SELECT * FROM pop_plus
ORDER BY column;